In [ ]:
!pip -q install ffmpeg-python
import ffmpeg

from IPython.display import Audio,HTML,Javascript
from google.colab.output import eval_js
from base64 import b64decode
from io import BytesIO

from scipy.io.wavfile import read as wav_read

from google.colab import drive
drive.mount('/root/gdrive')


import matplotlib.pyplot as plt

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

AUDIO_SCRIPT = """
var my_div = document.createElement("DIV");
var my_btn = document.createElement("BUTTON");
var t0 = document.createTextNode("________");
var my_btn1 = document.createElement("BUTTON");
var t01 = document.createTextNode("________");
var my_btn2 = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");
var t1 = document.createTextNode("recording");
var t2 = document.createTextNode("1");

my_btn.appendChild(t);
my_btn1.appendChild(t1);
my_btn2.appendChild(t2);
my_div.appendChild(my_btn);
my_div.appendChild(t0);
my_div.appendChild(my_btn1);
my_div.appendChild(t01);
my_div.appendChild(my_btn2);
document.body.appendChild(my_div);
my_btn1.onclick=function()
{
  if (my_btn1.innerText=="recording")
  {
     recorder.stream.getAudioTracks().forEach(function(track) {track.enabled = false;});
     my_btn1.innerText="silent";
     my_btn2.innerText=parseInt(my_btn2.innerText)+1;
     sleep(1000).then(() => {recorder.stream.getAudioTracks().forEach(function(track) {track.enabled = true;});
    my_btn1.innerText="recording"});
  }
};

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  //firefox only surpport: "audio/webm", "audio/ogg", "audio/webm;codecs=opus"
  //chrome surpport: "audio/webm;codecs=pcm"
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    bitsPerSecond: 8820,
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

"""

def fix_riff_header(binary):
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]
  return riff

def record(sec=3):
  display(Javascript(AUDIO_SCRIPT))
  data = eval_js("data")

  #display(Javascript(RECORD))
  #data = eval_js('record(%d)' % (sec*1000))

  binary = b64decode(data.split(',')[1])
  binary = fix_riff_header(binary)
  with open('audio.wav','wb') as f:
    f.write(binary)

  sr, audio = wav_read(BytesIO(binary))
  plt.figure(figsize=(20,10))
  plt.plot(audio)
  #plt.show()

  return ""

Mounted at /root/gdrive


In [ ]:
record()
display(Audio("/content/audio.wav"))
!ffmpeg -i audio.wav -vn -acodec pcm_s16le -ar 8820 -ac 1 /root/gdrive/MyDrive/ColabData/YonaVox/ac_audio/`date +%H%M%S|cut -b 2-6`.wav >/dev/null 2>&1